In [12]:
import csv
import pandas as pd

In [22]:
with open('boletins (08-07 a 19-08).csv', 'r') as arquivo:
    df = pd.read_csv(arquivo, sep= ';')

Retira as colunas em que todos os elementos são nan

In [23]:
df = df.dropna(axis=1, how='all')

#  converte todos os valores para números
df['ocupacao_leitos'] = df['ocupacao_leitos'] .apply(pd.to_numeric, errors='coerce')

print(df.head(2))

         Data  id                  regional                      data  \
0  08/07/2021   1              Estado de SC  2021-07-08T03:00:00.000Z   
1  08/07/2021   2  Alto Uruguai Catarinense  2021-07-08T03:00:00.000Z   

   var_media_movel    rt  letalidade  incidencia  ocupacao_leitos  \
0           -16.88  0.89        1.60    14832.64            83.32   
1           -26.18  0.86        1.33    16865.10           100.00   

   vacinacao_d1  vacinacao_d2  letalidade_sc  incidencia_sc  fase_anterior  
0       40.2263       12.2734            1.6       14832.64            1.0  
1       43.3161       14.9591            1.6       14832.64            1.0  


In [30]:
pontuacaoVacinas = {
    1.0: {'eval': lambda a: a < 0.2},
    2.0: {'eval': lambda a: a > 0.2},
    3.1: { 'eval': lambda a: a > 0.3},
    3.2: {'eval': lambda a: a > 0.4},
    3.3: {'eval': lambda a: a > 0.5},
    4.0: {'eval': lambda a: a > 0.75},
    '': {'eval': lambda a: a > 1},
}

In [31]:
def pontuacaoRegiao(row):
    pontos = 0
    if row['rt'] <= 1:
        pontos += 5
    if row['ocupacao_leitos'] <= 60:
        pontos += 5         
    if row['var_media_movel'] <= 15:
        pontos += 5
    if row['letalidade'] <= row['letalidade_sc']:
        pontos += 2
    if row['incidencia'] <= row['incidencia_sc']:
        pontos += 2
    
    f = pontuacaoVacinas[row['fase_anterior']]['eval']
    if (f.eval(row['vacinacao_d2'])):
        pontos += 3

    return pontos

In [32]:
df['pontuacao'] = df.apply(pontuacaoRegiao, axis=1)
print(df.head(1))

AttributeError: 'function' object has no attribute 'eval'